# Predicción de Utilidades de Películas

En primer lugar se cargaron los archivos de datos.

El conjunto de entrada X corresponde a un dataset disperso o *sparse* lo que significa que tiene una gran cantidad de elementos iguales a cero. Por tanto es importante mantener esta matriz en formato disperso para usar menos memoria haciendo uso de compresión y usar algoritmos eficientes para estas matrices para distintas operaciones. 

En este caso se cargaron las matrices en formato csc_matrix (*Compressed Sparse Column matrix*) de forma de acelerar operaciones sobre columnas.

La entrada considera 145256 características. El conjunto de entrenamiento consta de 1147 ejemplos, el de validación 317 y el de pruebas 254.

In [23]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix
from scipy.io import mmread



print "Cargando set de entrenamiento..."
X = csc_matrix(mmread('train.x.mm'))
y = np.loadtxt('train.y.dat')
print "Cargando set de validacion..."
Xv = csc_matrix(mmread('dev.x.mm'))
yv = np.loadtxt('dev.y.dat')
print "Cargando set de prueba..."
Xt = csc_matrix(mmread('test.x.mm'))
yt = np.loadtxt('test.y.dat')
print "Conjuntos cargados"

print X.shape
print Xv.shape
print Xt.shape

Cargando set de entrenamiento...
Cargando set de validacion...
Cargando set de prueba...
Conjuntos cargados
(1147, 145256)
(317, 145256)
(254, 145256)


Para obtener una meta inicial se aplicó directamente sobre el conjunto una regresión lineal sin ninguna preparación de los datos de forma de obtener un coeficiente de determinación inicial sobre el conjunto de pruebas el cual mejorar. Se puede ver que al aplicar este enfoque se produce un sobre-ajuste.

$R^2$ sobre el conjunto de entrenamiento es igual o muy cercano a $1$, sin embargo, tanto sobre el conjunto de validación como del conjunto de pruebas éste se acerca solo a $0.60$.

In [9]:
import sklearn.linear_model as lm

linreg = lm.LinearRegression(fit_intercept=False)
linreg.fit(X,y)

print "training R2=%f"%linreg.score(X,y)
print "validate R2=%f"%linreg.score(Xv,yv)
print "test R2=%f"%linreg.score(Xt,yt)

training R2=1.000000
validate R2=0.612864
test R2=0.590315


Se probaron distintos enfoques. Los más destacables siendo:

- Umbral de varianza
- Forward Stepwise Selection
- Regresión Lasso

No obstante ningún enfoque logro una mejora substancial sobre la regresión lineal anteriormente vista.

## Umbral de Varianza

Umbral de varianza es una tipo de selección de características básico que remueve aquellas características de baja varianza con la idea de que estas no tendrán tanta influencia sobre la solución. Un caso extremo es cuando una característica tenga varianza igual a cero, se puede remover con seguridad puesto que todos sus valores son iguales.

Se probaron umbrales de $0.5$, $0.1$, $0.01$ y $0.001$ y se calcularon los coeficientes de determinación. Se puede ver que se puede remover una gran cantidad de variables (pasando de 145256 a 58685) sin tener un cambio significativo de $R^2$ sobre todos los conjuntos. Sin embargo al reducir por este criterio aún más características el $R^2$ se reduce enormemente sobre los conjuntos de validación y entrenamiento.


In [19]:
import sklearn.feature_selection as fs

selector = fs.VarianceThreshold(threshold=0.5)
Xn = selector.fit_transform(X)
Xnt = selector.transform(Xt); Xnv = selector.transform(Xv)
linreg.fit(Xn,y)
print Xn.shape
print "training R2=%f"%linreg.score(Xn,y)
print "validate R2=%f"%linreg.score(Xnv,yv)
print "test R2=%f"%linreg.score(Xnt,yt)

selector = fs.VarianceThreshold(threshold=0.1)
Xn = selector.fit_transform(X)
Xnt = selector.transform(Xt); Xnv = selector.transform(Xv)
linreg.fit(Xn,y)
print Xn.shape
print "training R2=%f"%linreg.score(Xn,y)
print "validate R2=%f"%linreg.score(Xnv,yv)
print "test R2=%f"%linreg.score(Xnt,yt)

selector = fs.VarianceThreshold(threshold=0.01)
Xn = selector.fit_transform(X)
Xnt = selector.transform(Xt); Xnv = selector.transform(Xv)
linreg.fit(Xn,y)
print Xn.shape
print "training R2=%f"%linreg.score(Xn,y)
print "validate R2=%f"%linreg.score(Xnv,yv)
print "test R2=%f"%linreg.score(Xnt,yt)

selector = fs.VarianceThreshold(threshold=0.001)
Xn = selector.fit_transform(X)
Xnt = selector.transform(Xt); Xnv = selector.transform(Xv)
linreg.fit(Xn,y)
print Xn.shape
print "training R2=%f"%linreg.score(Xn,y)
print "validate R2=%f"%linreg.score(Xnv,yv)
print "test R2=%f"%linreg.score(Xnt,yt)


(1147, 1087)
training R2=0.991065
validate R2=-1.346798
test R2=-0.464357
(1147, 6214)
training R2=1.000000
validate R2=0.567887
test R2=0.553110
(1147, 58685)
training R2=1.000000
validate R2=0.611514
test R2=0.593959
(1147, 145222)
training R2=1.000000
validate R2=0.612862
test R2=0.590313


## Forward Stepwise Selection

Se aplicó también FSS. Sin embargo, por eficiencia no se aplicó directamente sobre todo el conjunto de características. En lugar de eso, en cada iteración se tomaban aleatoriamente 10 características aún no integradas en el modelo y se integraba la mejor. 

Se probó el puntaje de coeficientes del modelo

El proceso se detenía si encontraba un modelo con $R^2$ mayor a $0.6$ sobre el conjunto de pruebas (notar que solo se aprovechaba como criterio de parada, no para la construcción del modelo). Además el proceso también se detenía luego de la inclusión de 1000 características por temas de eficiencia pues en este punto cada generación del modelo tomaba más tiempo.

### Coeficientes del modelo

Al hacer uso de los coeficientes del modelo al integrar una característica se esperaba llegar a un modelo que ingresara aquellas características con influencia sobre la respuesta.

Así el puntaje era calculado como: 
```python
#indexes los indices de las características ingresadas al modelo en esta iteración más el candidato
score = abs(model.coef_[indexes.index(candidate)])
```
Este enfoque provoco un fuerte sobre-ajuste del conjunto de entrenamiento.

```
#Salida de FSS usando coeficientes del modelo
...
R2test = -0.661601
selected = 132655 ...
totalvars=1000, R2train = 0.999654, R2val = -0.974732
R2test = -0.673779
selected = 110725 ...
totalvars=1001, R2train = 0.999654, R2val = -0.965488
R2test = -0.658075
training R2=0.999654
validate R2=-0.965488
test R2=-0.658075
```
Se puede ver que habiendo agregado un gran número de características al modelo, aunque el valor de $R^2$ era alto en el conjunto de entrenamiento, se volvió negativo en el conjunto de validación y en el conjunto de pruebas lo cual muestra que este enfoque en realidad no funciona.

## Lasso

Finalmente se decidió probar con regresión Lasso. La razón para probar esta regresión es que el criterio de umbral de varianza parecia sugerir que al eliminar coeficientes se lograba un modelo correcto. 

Lasso elimina coeficientes de aquellas características que podrían catalogarse de menor importancia, es decir, que tienen menor influencia en la respuesta.

Se hizo una búsqueda semi-manual del parámetro $\alpha$ que maximizara $R^2$ sobre el conjunto de pruebas. La búsqueda se realizó en el rango $[10^{100},10^0]$

```python
#alphas_ = np.logspace(100,10,base=10)
#alphas_ = np.logspace(10,0,base=10,num=10)
#alphas_ = np.logspace(3,5,base=10,num=10)
#alphas_ = np.arange(1000.,4000.,500).tolist()
alphas_ = np.arange(1000.,4000.,500).tolist()
model = lm.Lasso(fit_intercept = False)
for a in alphas_:
	print "alpha=%f"%a
	model.set_params(alpha=a)
	model.fit(X,y)
	print "training R2=%f"%model.score(X,y)
	print "validate R2=%f"%model.score(Xv,yv)
	print "test R2=%f"%model.score(Xt,yt)
```



```
...
alpha=1500.000000
training R2=0.999961
validate R2=0.536099
test R2=0.540964
alpha=2000.000000
training R2=0.999931
validate R2=0.543443
test R2=0.546293
alpha=2500.000000
training R2=0.999892
validate R2=0.548085
test R2=0.543148
...
```

Se puede ver que el máximo valor de $R^2$ que obtiene Lasso sobre el conjunto de pruebas es de $R^2 = 0.546$ para $alpha = 2000$ lo cual esta por debajo del modelo de regresión lineal. Es cuestionable que $R^2$ cambie significativamente en la vecindad del valor de $\alpha$.

## Conclusiones

Se probaron diferentes enfoques los que obtuvieron diferentes valores para el coeficiente de determinación. 

Al aplicar un modelo de regresión lineal directamente sobre el modelo se obtuvo un coeficiente de determinación base de $0.590$ sobre el conjunto de pruebas. Se intentó mejorar sobre este con distintos enfoques.

El primer enfoque de Umbral de Varianza consistía en eliminar aquellas características con una varianza menor a cierto umbral. Al aplicar este criterio se logró reducir el número de características de 145265 a 58685, mejorando el valor de $R^2$ sobre el conjunto de pruebas marginalmente a $R^2 = 0.594$. 

Un problema es la eficiencia de la solución debido al tamaño de la entrada, con esto se puede ver que incluso reduciendo el número de características significativamente se puede mantener un $R^2$ similar a usar todo el conjunto para construir el modelo. 

El segundo enfoque consistió en aplicar FSS eligiendo aquella característica con el mayor coeficiente asociado. Este enfoque consiguió un mal resultado con un $R^2$ que era retornado como negativo por lo que probablemente el modelo resultante podría considerarse invalido.

El tercero enfoque fue aplicar regresión Lasso. Lasso no logró mejorar el coeficiente de determinación base encontrando un $R^2 = 0.546$ para $\alpha = 2000$.

Probablemente se deba trabajar con una combinación de otros enfoques distintos y con el umbral de varianza que es el que obtuvo resultados decentes. Quizás sea razonable probar otros criterios para la selección de características en Forward Stepwise Selection o probar otros enfoques de selección de características.
